<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/Catboost_20240825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 10.4 MB/s eta 0:00:00


In [4]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [5]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [30]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train6.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test6.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [31]:
df_train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚,FamilySize,IncomePerAge,IncomePerFamilySize,IncomePerTrips,PersonTripsProduct,IncomePerChild,LogMonthlyIncome,PitchFollowupInteraction,TripsPerAge,IncomeStarInteraction,FollowupEfficiency,IncomePerPerson,StarPerCityTier,IncomePerStar,LuxuryAffinity,FollowupPerTrip
0,0,50.00000,1,2,15.0,0,1.0,4.0,3.0,5.0,1,4,253,1.0,0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,5.060000,126.5,50.6,5.0,inf,5.537334,60.0,0.100000,759.0,16.0,126.5,1.500000,84.333333,253.0,0.80
1,1,56.00000,0,1,14.0,0,1.0,4.0,3.0,2.0,1,4,404,0.0,1,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,7.214286,404.0,202.0,2.0,inf,6.003887,56.0,0.035714,1212.0,16.0,202.0,3.000000,134.666667,404.0,2.00
2,2,41.13181,1,1,10.0,1,1.0,3.0,3.0,4.0,0,4,278,1.0,0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,6.758759,139.0,69.5,4.0,inf,5.631212,30.0,0.097248,834.0,12.0,139.0,3.000000,92.666667,278.0,0.75
3,3,37.00000,1,2,18.0,1,1.0,3.0,4.0,1.0,0,5,326,0.0,1,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,8.810811,326.0,326.0,1.0,inf,5.789960,54.0,0.027027,1304.0,15.0,163.0,2.000000,81.500000,326.0,3.00
4,4,48.00000,0,3,17.0,1,1.0,3.0,4.0,4.0,0,4,258,1.0,1,0.0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.375000,258.0,64.5,4.0,inf,5.556828,51.0,0.083333,1032.0,12.0,129.0,1.333333,64.500000,258.0,0.75


In [33]:
# Config cell
target = "ProdTaken"

In [165]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['IncomePerAge','FamilySize','IncomePerFamilySize','IncomePerTrips','PersonTripsProduct','IncomePerChild','LogMonthlyIncome','PitchFollowupInteraction','TripsPerAge',
'IncomeStarInteraction','FollowupEfficiency','IncomePerPerson','StarPerCityTier','IncomePerStar','LuxuryAffinity','FollowupPerTrip']
numerical_features = default_numerical_features + added_numerical_features



#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=['id',target,'IncomePerStar','IncomePerPerson','IncomePerFamilySize','IncomeStarInteraction','NumberOfFollowups',
'NumberOfTrips','PersonTripsProduct','IncomePerAge','PitchFollowupInteraction','IncomePerChild'
,'Occupation_2','FollowupPerTrip','ProductPitched_STANDARD','NumberOfPersonVisiting','Occupation_0',
'MaritalStatus_離婚','MaritalStatus_結婚','MaritalStatus_未婚',
'Designation_VP','Designation_AVP','ProductPitched_SUPERDELUXE',
'ProductPitched_KING','Occupation_1','NumberOfChildren','Designation_Senior Manager',
            'DurationOfPitch','OwnCar','FamilySize','PitchSatisfactionScore','PreferredPropertyStar','CityTier','HasChildren']
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar', 'IncomePerAge', 'FamilySize', 'IncomePerFamilySize', 'IncomePerTrips', 'PersonTripsProduct', 'IncomePerChild', 'LogMonthlyIncome', 'PitchFollowupInteraction', 'TripsPerAge', 'IncomeStarInteraction', 'FollowupEfficiency', 'IncomePerPerson', 'StarPerCityTier', 'IncomePerStar', 'LuxuryAffinity', 'FollowupPerTrip']
categorical_features ['TypeofContact', 'CityTier', 'Gender', 'PreferredPropertyStar', 'Passport', 'PitchSatisfactionScore', 'ProdTaken', 'HasChildren', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior Manager', 'Designation_VP', 'MaritalStatus_未婚', 'MaritalStatus_独身', 'MaritalStatus_結婚', 'MaritalStatus_離婚']


In [166]:
cols_category=categorical_features

In [161]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [157]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    'depth':1,
    'l2_leaf_reg' : 8,
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [167]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [111]:
!pip install shap

In [112]:
import shap

In [168]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []
shap_values_list = []

list_feature_importances = []  # 特徴量重要度を格納するリスト

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x[features].iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x[features].iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

    # 特徴量の重要度を追加
    feature_importances = model_cat.get_feature_importance()
    list_feature_importances.append(feature_importances)

# AUCの出力
print("Mean AUC:", np.mean(list_metrics_auc))
print("Mean F1 Score:", np.mean(list_metrics_f1))
print("Median Cutoff:", np.median(list_cutoff))

# 特徴量重要度の集計
mean_feature_importances = np.mean(list_feature_importances, axis=0)

# 結果の表示
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Mean Importance': mean_feature_importances
}).sort_values(by='Mean Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

0:	test: 0.7186957	best: 0.7186957 (0)	total: 1.21ms	remaining: 1.2s
100:	test: 0.8390635	best: 0.8397659 (93)	total: 218ms	remaining: 1.94s
200:	test: 0.8445067	best: 0.8451254 (193)	total: 543ms	remaining: 2.16s
300:	test: 0.8445067	best: 0.8457358 (234)	total: 864ms	remaining: 2.01s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.845735786
bestIteration = 234

Shrink model to first 235 iterations.
Fold: 1, AUC: 0.8457357859531773, f1 score: 0.7288267288267287 Threshold: 0.24380731809751574
0:	test: 0.6454515	best: 0.6454515 (0)	total: 4.7ms	remaining: 4.69s
100:	test: 0.8485368	best: 0.8485368 (100)	total: 287ms	remaining: 2.55s
200:	test: 0.8477676	best: 0.8500418 (121)	total: 586ms	remaining: 2.33s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.850041806
bestIteration = 121

Shrink model to first 122 iterations.
Fold: 2, AUC: 0.850041806020067, f1 score: 0.7447789115646258 Threshold: 0.28927644949898185
0:	test: 0.7040353	best: 0.7040353 

In [81]:
0.8457882834182102


SyntaxError: invalid syntax (<ipython-input-81-76ea1f53a6c0>, line 1)

In [ ]:
0.8437365146029495

In [170]:
import pickle

In [172]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v8{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [171]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x7c8fc46c2560>, Proba: [0.01791486 0.04611592 0.04832069 ... 0.1110678  0.05172635 0.00159956]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fc4195d80>, Proba: [0.03375053 0.09311165 0.10381576 ... 0.23635364 0.11744564 0.00507842]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fdc3974f0>, Proba: [0.04905228 0.14566056 0.15609819 ... 0.36443725 0.17862555 0.00677712]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fc46c1fc0>, Proba: [0.06646801 0.19922438 0.20360454 ... 0.49057089 0.23321065 0.00865924]
Model: <catboost.core.CatBoostClassifier object at 0x7c8fc42a0460>, Proba: [0.08176744 0.23573588 0.25740433 ... 0.61703428 0.29572738 0.01129979]


In [173]:
count = 0

In [174]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)